In [1]:
!pip install yfinance pandas ta

In [2]:
import yfinance as yf
import pandas as pd

In [3]:
ticker = "NVDA"
data = yf.download(ticker, period="max", interval="1d")  # 1 year of daily data

[*********************100%***********************]  1 of 1 completed


In [4]:
data

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,,
1999-01-22,0.037615,0.041016,0.048828,0.038802,0.043750,2714688000
1999-01-25,0.041556,0.045313,0.045833,0.041016,0.044271,510480000
1999-01-26,0.038331,0.041797,0.046745,0.041146,0.045833,343200000
1999-01-27,0.038212,0.041667,0.042969,0.039583,0.041927,244368000
1999-01-28,0.038092,0.041536,0.041927,0.041276,0.041667,227520000
...,...,...,...,...,...,...
2024-12-31,134.289993,134.289993,138.070007,133.830002,138.029999,155659200
2025-01-02,138.309998,138.309998,138.880005,134.630005,136.000000,198247200


In [5]:
# Simple Moving Averages
data['MA10'] = data['Close'].rolling(window=10).mean()  # 10-day MA
data['MA50'] = data['Close'].rolling(window=50).mean()  # 50-day MA

# Exponential Moving Averages
data['EMA10'] = data['Close'].ewm(span=10, adjust=False).mean()  # 10-day EMA
data['EMA50'] = data['Close'].ewm(span=50, adjust=False).mean()  # 50-day EMA

In [6]:
from ta.momentum import RSIIndicator

# Add RSI (default is 14-day period)
data['RSI'] = RSIIndicator(data['Close']['NVDA']).rsi()

In [7]:
from ta.volatility import BollingerBands

# Calculate Bollinger Bands
bb_indicator = BollingerBands(data['Close']['NVDA'])
data['BB_upper'] = bb_indicator.bollinger_hband()
data['BB_lower'] = bb_indicator.bollinger_lband()
data['BB_middle'] = bb_indicator.bollinger_mavg()

In [8]:
from ta.volatility import AverageTrueRange

# Calculate ATR
data['ATR'] = AverageTrueRange(data['High']['NVDA'], data['Low']['NVDA'], data['Close']['NVDA']).average_true_range()

In [9]:
# Flatten MultiIndex columns
data.columns = ['_'.join(col).strip() for col in data.columns.values]

In [10]:
data

,Adj Close_NVDA,Close_NVDA,High_NVDA,Low_NVDA,Open_NVDA,Volume_NVDA,MA10_,MA50_,EMA10_,EMA50_,RSI_,BB_upper_,BB_lower_,BB_middle_,ATR_
Date,,,,,,,,,,,,,,,
1999-01-22,0.037615,0.041016,0.048828,0.038802,0.043750,2714688000,NaN,NaN,0.041016,0.041016,NaN,NaN,NaN,NaN,0.000000
1999-01-25,0.041556,0.045313,0.045833,0.041016,0.044271,510480000,NaN,NaN,0.041797,0.041185,NaN,NaN,NaN,NaN,0.000000
1999-01-26,0.038331,0.041797,0.046745,0.041146,0.045833,343200000,NaN,NaN,0.041797,0.041209,NaN,NaN,NaN,NaN,0.000000
1999-01-27,0.038212,0.041667,0.042969,0.039583,0.041927,244368000,NaN,NaN,0.041774,0.041227,NaN,NaN,NaN,NaN,0.000000
1999-01-28,0.038092,0.041536,0.041927,0.041276,0.041667,227520000,NaN,NaN,0.041730,0.041239,NaN,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,134.289993,134.289993,138.070007,133.830002,138.029999,155659200,135.328998,139.890399,136.460739,136.451693,45.173350,146.138021,128.158976,137.148499,5.070568
2025-01-02,138.309998,138.309998,138.880005,134.630005,136.000000,198247200,136.120998,139.782399,136.796968,136.524567,51.412395,145.945212,128.156785,137.050999,5.036243
2025-01-03,144.470001,144.470001,144.899994,139.729996,140.009995,229322500,137.676997,139.799999,138.192065,136.836153,59.094009,145.793858,128.241140,137.017499,5.147225


In [11]:
tickers = ["NVDA", "AMD", "INTC", "^GSPC", "^IXIC"]

# Fetch data for each ticker
data_dict = {}
for ticker in tickers:
    data_dict[ticker] = yf.download(ticker, period="max", interval="1d")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
# Extract relevant columns and rename them
for ticker in tickers:
    data[f"{ticker}_Close"] = data_dict[ticker]['Close']

In [25]:
from dotenv import load_dotenv
import os
import requests

# Load environment variables from .env file
load_dotenv()

# Access the API key
FRED_API_KEY = os.getenv("FRED_API_KEY")

In [27]:
interest_endpoint = f"https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&api_key={FRED_API_KEY}&file_type=json"
interest_response = requests.get(interest_endpoint)

In [29]:
gdp_endpoint = f"https://api.stlouisfed.org/fred/series/observations?series_id=GDP&api_key={FRED_API_KEY}&file_type=json"
gdp_response = requests.get(gdp_endpoint)

In [31]:
start_date = data.index.min().strftime('%Y-%m-%d')
end_date = data.index.max().strftime('%Y-%m-%d')

print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 1999-01-22
End Date: 2025-01-07


In [33]:
gdp_data = gdp_response.json()

In [35]:
gdp_df = pd.DataFrame(gdp_data['observations'])
gdp_df['date'] = pd.to_datetime(gdp_df['date'])
gdp_df['value'] = pd.to_numeric(gdp_df['value'], errors='coerce')
gdp_df = gdp_df.rename(columns={'date': 'Date', 'value': 'GDP'}).set_index('Date')

In [37]:
gdp_daily = gdp_df.resample('D').ffill()

In [39]:
gdp_daily = gdp_daily[start_date:end_date]

In [41]:
gdp_daily

,realtime_start,realtime_end,GDP
Date,,,
1999-01-22,2024-12-19,2024-12-19,9411.682
1999-01-23,2024-12-19,2024-12-19,9411.682
1999-01-24,2024-12-19,2024-12-19,9411.682
1999-01-25,2024-12-19,2024-12-19,9411.682
1999-01-26,2024-12-19,2024-12-19,9411.682
...,...,...,...
2024-06-27,2024-12-19,2024-12-19,29016.714
2024-06-28,2024-12-19,2024-12-19,29016.714
2024-06-29,2024-12-19,2024-12-19,29016.714


In [43]:
interest_data = interest_response.json()

In [45]:
interest_df = pd.DataFrame(interest_data['observations'])
interest_df['date'] = pd.to_datetime(interest_df['date'])
interest_df['value'] = pd.to_numeric(interest_df['value'], errors='coerce')
interest_df = interest_df.rename(columns={'date': 'Date', 'value': 'Interest_Rate'}).set_index('Date')

In [47]:
interest_daily = interest_df.resample('D').ffill()

In [49]:
interest_daily = interest_daily[start_date:end_date]

In [51]:
interest_daily

,realtime_start,realtime_end,Interest_Rate
Date,,,
1999-01-22,2025-01-07,2025-01-07,4.63
1999-01-23,2025-01-07,2025-01-07,4.63
1999-01-24,2025-01-07,2025-01-07,4.63
1999-01-25,2025-01-07,2025-01-07,4.63
1999-01-26,2025-01-07,2025-01-07,4.63
...,...,...,...
2024-11-27,2025-01-07,2025-01-07,4.64
2024-11-28,2025-01-07,2025-01-07,4.64
2024-11-29,2025-01-07,2025-01-07,4.64


In [53]:
# Merge GDP and Interest Rate into data
data = data.merge(gdp_daily, how='left', left_index=True, right_index=True)
data = data.merge(interest_daily, how='left', left_index=True, right_index=True)

In [55]:
data

,Adj Close_NVDA,Close_NVDA,High_NVDA,Low_NVDA,Open_NVDA,Volume_NVDA,MA10_,MA50_,EMA10_,EMA50_,...,AMD_Close,INTC_Close,^GSPC_Close,^IXIC_Close,realtime_start_x,realtime_end_x,GDP,realtime_start_y,realtime_end_y,Interest_Rate
Date,,,,,,,,,,,,,,,,,,,,,
1999-01-22,0.037615,0.041016,0.048828,0.038802,0.043750,2714688000,NaN,NaN,0.041016,0.041016,...,10.781250,32.218750,1225.189941,2338.879883,2024-12-19,2024-12-19,9411.682,2025-01-07,2025-01-07,4.63
1999-01-25,0.041556,0.045313,0.045833,0.041016,0.044271,510480000,NaN,NaN,0.041797,0.041185,...,10.718750,32.656250,1233.979980,2369.310059,2024-12-19,2024-12-19,9411.682,2025-01-07,2025-01-07,4.63
1999-01-26,0.038331,0.041797,0.046745,0.041146,0.045833,343200000,NaN,NaN,0.041797,0.041209,...,11.000000,34.234375,1252.310059,2433.409912,2024-12-19,2024-12-19,9411.682,2025-01-07,2025-01-07,4.63
1999-01-27,0.038212,0.041667,0.042969,0.039583,0.041927,244368000,NaN,NaN,0.041774,0.041227,...,10.625000,33.187500,1243.170044,2407.139893,2024-12-19,2024-12-19,9411.682,2025-01-07,2025-01-07,4.63
1999-01-28,0.038092,0.041536,0.041927,0.041276,0.041667,227520000,NaN,NaN,0.041730,0.041239,...,11.437500,34.296875,1265.369995,2477.340088,2024-12-19,2024-12-19,9411.682,2025-01-07,2025-01-07,4.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,134.289993,134.289993,138.070007,133.830002,138.029999,155659200,135.328998,139.890399,136.460739,136.451693,...,120.790001,20.049999,5881.629883,19310.789062,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-02,138.309998,138.309998,138.880005,134.630005,136.000000,198247200,136.120998,139.782399,136.796968,136.524567,...,120.629997,20.219999,5868.549805,19280.789062,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-03,144.470001,144.470001,144.899994,139.729996,140.009995,229322500,137.676997,139.799999,138.192065,136.836153,...,125.370003,20.559999,5942.470215,19621.679688,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
data = data.drop(columns=['realtime_start_x', 'realtime_end_x', 'realtime_start_y', 'realtime_end_y'], errors='ignore')

In [59]:
data.to_csv('uncleaned.csv')

In [61]:
data

,Adj Close_NVDA,Close_NVDA,High_NVDA,Low_NVDA,Open_NVDA,Volume_NVDA,MA10_,MA50_,EMA10_,EMA50_,...,BB_lower_,BB_middle_,ATR_,NVDA_Close,AMD_Close,INTC_Close,^GSPC_Close,^IXIC_Close,GDP,Interest_Rate
Date,,,,,,,,,,,,,,,,,,,,,
1999-01-22,0.037615,0.041016,0.048828,0.038802,0.043750,2714688000,NaN,NaN,0.041016,0.041016,...,NaN,NaN,0.000000,0.041016,10.781250,32.218750,1225.189941,2338.879883,9411.682,4.63
1999-01-25,0.041556,0.045313,0.045833,0.041016,0.044271,510480000,NaN,NaN,0.041797,0.041185,...,NaN,NaN,0.000000,0.045313,10.718750,32.656250,1233.979980,2369.310059,9411.682,4.63
1999-01-26,0.038331,0.041797,0.046745,0.041146,0.045833,343200000,NaN,NaN,0.041797,0.041209,...,NaN,NaN,0.000000,0.041797,11.000000,34.234375,1252.310059,2433.409912,9411.682,4.63
1999-01-27,0.038212,0.041667,0.042969,0.039583,0.041927,244368000,NaN,NaN,0.041774,0.041227,...,NaN,NaN,0.000000,0.041667,10.625000,33.187500,1243.170044,2407.139893,9411.682,4.63
1999-01-28,0.038092,0.041536,0.041927,0.041276,0.041667,227520000,NaN,NaN,0.041730,0.041239,...,NaN,NaN,0.000000,0.041536,11.437500,34.296875,1265.369995,2477.340088,9411.682,4.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,134.289993,134.289993,138.070007,133.830002,138.029999,155659200,135.328998,139.890399,136.460739,136.451693,...,128.158976,137.148499,5.070568,134.289993,120.790001,20.049999,5881.629883,19310.789062,NaN,NaN
2025-01-02,138.309998,138.309998,138.880005,134.630005,136.000000,198247200,136.120998,139.782399,136.796968,136.524567,...,128.156785,137.050999,5.036243,138.309998,120.629997,20.219999,5868.549805,19280.789062,NaN,NaN
2025-01-03,144.470001,144.470001,144.899994,139.729996,140.009995,229322500,137.676997,139.799999,138.192065,136.836153,...,128.241140,137.017499,5.147225,144.470001,125.370003,20.559999,5942.470215,19621.679688,NaN,NaN
